In [1]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
%matplotlib qt

In [2]:
#solving the wave equation: u_t + u u_x=\nu u_xx

nu=0.001; #viscosity
N=1000; # number of grid boxes
L=1; # length of the domain
x=np.arange(0.5,N+0.5)/N*L-L/2; 

dx=x[1]-x[0];

# initial conditions, a Gaussian
u0=np.exp(-((x+L/3)/(L/10))**2/2);
# Courant stability requires C_adv<1 and C_diff<0.5
dt_adv=dx/np.max(u0); 
dt_diff=0.5*dx**2/nu;
dt=0.1*np.min((dt_adv,dt_diff)); #time step, using prefactor of 0.1 to make sure you're stable.
total_time=1; #simulation time
Nt=int(np.fix(total_time/dt)); # number of timesteps
# define wavenumbers k
k0x=2*np.pi/L;
k = np.concatenate((np.arange(0, N/2 + 1), np.arange(-N/2 + 1, 0)))*k0x;
k2=k*k;
##
u=u0; #set initial conditions
u_burger = u0



fig,ax = plt.subplots()

line1 = ax.plot(x, u, label='Pseudo-Spectral Method')[0]
line2 = ax.plot(x, u_burger, label='Upwind Scheme')[0]
ax.legend()

def update(frame):
    global u,u_hat,u_hat_t,u_hat_t_prev
    global u_burger, u_burger_t, uxx, u_pos, u_neg, u_burger_t_prev
    time=frame*dt;
    line1.set_ydata(u);
    line2.set_ydata(u_burger)
    ax.set_ylim([-np.max(abs(u0)),np.max(abs(u0))]);
    u_hat = np.fft.fft(u); #hat variables are fourier transorms
    
    # Compute the tendency term in the spectral space here
    u_hat_t = -1j * k * np.fft.fft(u**2 / 2) - nu * k**2 * u_hat

    # Finite difference
    u_xx = np.zeros_like(u_burger)
    u_xx[1:-1]=(u_burger[2:]-2*u_burger[1:-1] + u_burger[:-2])/dx**2;
    u_xx[0] = (u_burger[1] - 2 * u_burger[0] + u_burger[-1])/dx**2 #left boundary
    u_xx[-1]= (u_burger[0] - 2 * u_burger[-1] + u_burger[-2])/dx**2; #right boundary

    # Upwind
    u_pos_x = np.zeros_like(u_burger)
    u_pos_x[:-1] = (u_burger[1:] - u_burger[:-1])/dx
    u_pos_x[-1] = (u_burger[0] - u_burger[-1])/dx

    # Downwind
    u_neg_x = np.zeros_like(u_burger)
    u_neg_x[1:] = (u_burger[1:] - u_burger[:-1])/dx
    u_neg_x[0] = (u_burger[0] - u_burger[-1])/dx

    u_pos = np.maximum(u_burger, 0)
    u_neg = np.minimum(u_burger, 0)
    uux = u_pos * u_pos_x + u_neg * u_neg_x
    
    # Forwards
    u_burger_t = nu*u_xx - uux

    # Adams-Bashforth's scheme goes here
    if frame == 0:
        u_hat_next = u_hat + u_hat_t * dt
        u_burger = u_burger + u_burger_t * dt
    else:
        u_hat_next = u_hat + dt * (3/2 * u_hat_t - 1/2 * u_hat_t_prev)
        u_burger = u_burger + dt * (3/2 * u_burger_t - 1/2 * u_burger_t_prev)
    # Compute u(x) from hat_u using IFFT here
    u = np.real(np.fft.ifft(u_hat_next))
    
    u_hat_t_prev = u_hat_t
    u_burger_t_prev = u_burger_t

    
    return line1, line2
    
anim = FuncAnimation(fig,update,Nt,interval = 10,repeat=False)